In [ ]:
# Rotary pendulum adapted from spherical pendulum code
# import libraries
import sympy as sym
import numpy as np

from IPython.display import display #for pretty printing

# system parameters
X0,Y0, Z0 = sym.symbols(['X0','Y0', 'Z0']) # fixed position of Arm
g = sym.symbols('g')
mP,mA = sym.symbols(['mP','mA']) # mass of links
La,Lp = sym.symbols(['La','Lp']) # length of links
InA,InP = sym.symbols(['InA','InP']) # moment of intertia of links

# generalized coordinates
th1,ph1,th2,ph2 = sym.symbols(['\\theta_{1}','\\phi_{1}','\\theta_{2}','\\phi_{2}']) #position
dth1,dph1, dth2, dph2 = sym.symbols(['\\dot{\\theta_{1}}','\\dot{\\phi_{1}}', '\\dot{\\theta_{2}}','\\dot{\\phi_{2}}']) #velocity
ddth1,ddph1, ddth2, ddph2 = sym.symbols(['\\ddot{\\theta_{1}}','\\ddot{\\phi_{1}}', '\\ddot{\\theta_{2}}','\\ddot{\\phi_{2}}']) #acceleration

# set up to make rotary pendulum
th1 = np.pi/2
th2 = -np.pi/2

# assuming not absolute angles
thr = th1+th2
phr = ph1+ph2

q = sym.Matrix([[ph1],[ph2]]) #group into matrices
dq = sym.Matrix([[dph1],[dph2]])
ddq = sym.Matrix([[ddph1],[ddph2]])

qt = ddq.T
display(qt) #display prints it as cool latex stuff

# positiions
## Link 1
x1 = La*sym.sin(th1)*sym.cos(ph1)/2
y1 = -La*sym.sin(th1)*sym.sin(ph1)/2
z1 = -La*sym.cos(th1)/2

x2 = x1*2 + La*sym.sin(thr)*sym.cos(phr)/2
y2 = y1*2 - La*sym.sin(thr)*sym.sin(phr)/2
z2 = z1*2 - Lp*sym.cos(thr)/2

rArm = sym.simplify(sym.Matrix([x1, y1, z1]))
rPen = sym.simplify(sym.Matrix([x2, y2, z2]))


# velocities
## linear velocities
drArm = rArm.jacobian(q)*dq
drPen = rPen.jacobian(q)*dq

#display(drArm, drPen)
## angular velocities

# energies
## kinetic
Tarm = 1/2 * mA * drArm.T*drArm
Tpen = 1/2 * mP * drPen.T*drPen
T = sym.Matrix([sym.simplify(Tarm+Tpen)])


## potential
Varm = mA*g*z1
Vpen = mP*g*z2
V = sym.Matrix([sym.simplify(Varm+Vpen)])

# Lagrangian
Lg1 = sym.zeros(1,len(q))
for i in range(len(q)):
    dT_ddq = sym.diff(T,dq[i]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = T.jacobian(q) # partial of T in q

# term 4
Lg4 = V.jacobian(q) # partial of U in q
print("Positions------------------------------------")
display(rArm, rPen)


In [ ]:
# Torque
# STEP 5: calculate generalized forces
# control torque
tau = sym.symbols('\\tau')

Qtau = sym.Matrix([[tau],[0]]) # Torque tau acts in the ph1 direction

Qall = Qtau

# put it all together
EOM = Lg1 - Lg3 + Lg4 - Qall.transpose()
display(Qall)

In [ ]:
import matplotlib.pyplot as plt
# Test
N = 100
# forces
h = 0.01
t = np.arange(0,N*h,h)
tau_arr = 1*np.sin(8*np.pi*t)


#initial conditions
ph10 = np.pi/16
ph20 = 0

dph10 = 0
dph20 = 0

#parameters
X0val = 0
Y0val = 0
Z0val = 0
laval = 1
lpval = 1

parameter_values = [(X0,X0val),(Y0,Y0val),(g,9.81),(mA,1),(mP,1),(La,laval),(Lp,lpval),(InA,0.08),(InP,0.08)]


ph1ar = [ph10]
ph2ar = [ph20]

dph1ar = [dph10]
dph2ar = [dph20]


for i in range(1,N):
    # substitute parameter values and previous conditions into the EOM 
    past = [(ph1,ph1ar[i-1]),(ph2,ph2ar[i-1]),  
            (dph1,dph1ar[i-1]),(dph2,dph2ar[i-1])]
    forces = [(tau,tau_arr[i-1])]
    EOM_sub = EOM.subs(parameter_values).subs(past).subs(forces)
    display(EOM_sub)
    if i == 5:
        print("Broke")
        break

    
    # solve for the acceleration
    acc = sym.solve(EOM_sub,[ddph1, ddph2])
    
    # integrate for the next velocity and position
    # print("ddph1:", acc[ddph1])
    # print("ddph2:", acc[ddph2])

    dph1ar.append(float(dph1ar[i-1]+h*acc[ddph1]))
    dph2ar.append(float(dph2ar[i-1]+h*acc[ddph2]))
    
    ph1ar.append(float(ph1ar[i-1]+h*dph1ar[i]))
    ph2ar.append(float(ph2ar[i-1]+h*dph2ar[i]))

print(ph1ar)

In [ ]:
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt

# Define your other symbols and parameters here...

# Initialize variables
N = 100
h = 0.01
t = np.arange(0, N * h, h)
tau_arr = 1 * np.sin(8 * np.pi * t)

# Plotting tau_arr against t
plt.figure(figsize=(10, 5))
plt.plot(t, tau_arr, label='Torque (τ)', color='b')
plt.title('Torque vs Time')
plt.xlabel('Time (s)')
plt.ylabel('Torque (τ)')
plt.grid()
plt.legend()
plt.show()

# Initial conditions
ph10 = np.pi / 16
ph20 = 0
dph10 = 0
dph20 = 0

# Parameters
X0val = 0
Y0val = 0
Z0val = 0
laval = 1
lpval = 1

parameter_values = [(X0, X0val), (Y0, Y0val), (g, 9.81), (mA, 1), (mP, 1), (La, laval), (Lp, lpval), (InA, 0.08), (InP, 0.08)]

ph1ar = [ph10]
ph2ar = [ph20]
dph1ar = [dph10]
dph2ar = [dph20]

# RK4 integration function
def rk4_step(ph1, ph2, dph1, dph2, h, i):
    # Substitute parameter values and previous conditions into the EOM
    past = [(ph1, ph1ar[i-1]), (ph2, ph2ar[i-1]), (dph1, dph1ar[i-1]), (dph2, dph2ar[i-1])]
    forces = [(tau, tau_arr[i])]
    EOM_sub = EOM.subs(parameter_values).subs(past).subs(forces)

    # Calculate k1
    acc1 = sym.solve(EOM_sub, [ddph1, ddph2])
    k1_ph1 = h * dph1
    k1_dph1 = h * float(acc1[ddph1].evalf())
    k1_ph2 = h * dph2
    k1_dph2 = h * float(acc1[ddph2].evalf())

    # Calculate k2
    acc2 = sym.solve(EOM_sub.subs({dph1: dph1 + 0.5 * k1_dph1, dph2: dph2 + 0.5 * k1_dph2}), [ddph1, ddph2])
    k2_ph1 = h * (dph1 + 0.5 * k1_dph1)
    k2_dph1 = h * float(acc2[ddph1].evalf())
    k2_ph2 = h * (dph2 + 0.5 * k1_dph2)
    k2_dph2 = h * float(acc2[ddph2].evalf())

    # Calculate k3
    acc3 = sym.solve(EOM_sub.subs({dph1: dph1 + 0.5 * k2_dph1, dph2: dph2 + 0.5 * k2_dph2}), [ddph1, ddph2])
    k3_ph1 = h * (dph1 + 0.5 * k2_dph1)
    k3_dph1 = h * float(acc3[ddph1].evalf())
    k3_ph2 = h * (dph2 + 0.5 * k2_dph2)
    k3_dph2 = h * float(acc3[ddph2].evalf())

    # Calculate k4
    acc4 = sym.solve(EOM_sub.subs({dph1: dph1 + k3_dph1, dph2: dph2 + k3_dph2}), [ddph1, ddph2])
    k4_ph1 = h * (dph1 + k3_dph1)
    k4_dph1 = h * float(acc4[ddph1].evalf())
    k4_ph2 = h * (dph2 + k3_dph2)
    k4_dph2 = h * float(acc4[ddph2].evalf())

    # Update the values
    ph1_next = ph1 + (k1_ph1 + 2 * k2_ph1 + 2 * k3_ph1 + k4_ph1) / 6
    dph1_next = dph1 + (k1_dph1 + 2 * k2_dph1 + 2 * k3_dph1 + k4_dph1) / 6
    ph2_next = ph2 + (k1_ph2 + 2 * k2_ph2 + 2 * k3_ph2 + k4_ph2) / 6
    dph2_next = dph2 + (k1_dph2 + 2 * k2_dph2 + 2 * k3_dph2 + k4_dph2) / 6

    return ph1_next, dph1_next, ph2_next, dph2_next

# Simulation loop
for i in range(1, N):
    ph1_next, dph1_next, ph2_next, dph2_next = rk4_step(ph1ar[i - 1], ph2ar[i - 1], dph1ar[i - 1], dph2ar[i - 1], h, i)
    
    ph1ar.append(ph1_next)
    dph1ar.append(dph1_next)
    ph2ar.append(ph2_next)
    dph2ar.append(dph2_next)

print(ph1ar)
